In [1]:
import networkx as nx
import os
from utils.global_rules import rules

dataDir = "../data/"
cacheDir = "../data/cache/"

读入图数据，G: 无向图，G_directed: 有向图


In [11]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

G = process.get_graph()

{'node_priority': {'Domain': 3, 'IP': 4, 'Cert': 4, 'Whois_Name': 2, 'Whois_Phone': 2, 'Whois_Email': 2, 'IP_C': 1, 'ASN': 1}, 'link_priority': {'r_cert': 4, 'r_subdomain': 4, 'r_request_jump': 4, 'r_dns_a': 4, 'r_whois_name': 3, 'r_whois_email': 3, 'r_whois_phone': 3, 'r_cert_chain': 2, 'r_cname': 2, 'r_asn': 1, 'r_cidr': 1}, 'jump_limit': {1: 0, 2: 1, 3: 2, 4: 3}, 'net_limit': {'small': {'node': 400, 'edge': 800}, 'medium': {'node': 800, 'edge': 1600}, 'large': {'node': 3000, 'edge': 6000}}, 'evidence': {'1': ['Domain_c58c149eec59bb14b0c102a0f303d4c20366926b5c3206555d2937474124beb9', 'Domain_f3554b666038baffa5814c319d3053ee2c2eb30d31d0ef509a1a463386b69845'], '2': ['IP_400c19e584976ff2a35950659d4d148a3d146f1b71692468132b849b0eb8702c', 'Domain_b10f98a9b53806ccd3a5ee45676c7c09366545c5b12aa96955cde3953e7ad058'], '3': ['Domain_24acfd52f9ceb424d4a2643a832638ce1673b8689fa952d9010dd44949e6b1d9', 'Domain_9c72287c3f9bb38cb0186acf37b7054442b75ac32324dfd245aed46a03026de1', 'Domain_717aa5778731a1

In [12]:
# Create a Directed Graph from the undirected one
G_directed = nx.DiGraph(G)

运行社区检测


In [3]:
import importlib

community_detection = importlib.import_module("utils.community_detection")
importlib.reload(community_detection)

# community_detection.get_community_for_all_groups(G)
community_detection.get_subgraph_for_all_communities(G)

根据以上社区检测算法的结果，选取团伙 [?] 进行分析。


In [ ]:
import json
import importlib

rules = importlib.import_module("utils.global_rules")
process = importlib.import_module("process")
importlib.reload(rules)
importlib.reload(process)

rules = rules.rules
# group_to_analyze = ["1", "3", "4"]
group_to_analyze = ["1", "2", "3", "4", "5"]
cacheDir = "../data/cache/"
subgraphDir = "../data/subgraph/"


def export_subgraph_to_json(graph, group_str: str):
    # transform the graph to directed graph
    # graph = graph.to_directed()
    # export the graph to json
    json.dump(
        nx.readwrite.json_graph.node_link_data(graph),
        open(subgraphDir + "group_" + group_str + ".json", "w"),
    )
    return


def analyse_group(group_str: str):
    print("analyse group: ", group_str)
    # get the evidence
    evidence = rules["evidence"][group_str]

    # # read the group
    # graph = nx.readwrite.json_graph.node_link_graph(
    #     json.load(open(cacheDir + "group_" + group_str + ".json"))
    # )
    # # print graph info
    # print("#nodes: ", graph.number_of_nodes())
    # print("#edges: ", graph.number_of_edges())
    # subgraph = process.get_subgrah(graph, evidence)

    subgraph = process.get_subgraph(G_directed, evidence)
    print("#nodes in subgraph: ", subgraph.number_of_nodes())
    print("#edges in subgraph: ", subgraph.number_of_edges())

    subgraph_filtered = process.filter_subgraph(
        subgraph,
    )
    process.set_core(subgraph_filtered)
    export_subgraph_to_json(subgraph_filtered, group_str)

    print()
    return


for group in group_to_analyze:
    analyse_group(group)

# analyse_group("1")